## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-10-10-50-00 +0000,missionlocal,A musical homecoming for Samuel Adams at Brava...,https://missionlocal.org/2025/10/a-musical-hom...
1,2025-10-10-10-47-33 +0000,bbc,Google may be forced to make changes to search...,https://www.bbc.com/news/articles/c98d7p8l9pro...
2,2025-10-10-10-38-06 +0000,wapo,Library director fired over LGBTQ+ books gets ...,https://www.washingtonpost.com/nation/2025/10/...
3,2025-10-10-10-37-29 +0000,nyt,Israel Says Cease-Fire Is in Effect in Gaza,https://www.nytimes.com/2025/10/10/world/middl...
4,2025-10-10-10-35-39 +0000,bbc,"Don't force drivers to use parking apps, says RAC",https://www.bbc.com/news/articles/ce847ngn7n8o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2340/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
82,trump,47
25,gaza,40
88,deal,36
21,israel,20
172,new,18


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
39,2025-10-10-09-06-09 +0000,nypost,Why Trump wasn’t awarded Nobel Peace Prize for...,https://nypost.com/2025/10/10/us-news/why-trum...,198
206,2025-10-09-20-54-59 +0000,nypost,Jubilant Trump vows Gaza peace deal will ‘move...,https://nypost.com/2025/10/09/us-news/jubilant...,188
248,2025-10-09-19-02-29 +0000,nypost,Rep. Andy Barr latest to nominate Trump for No...,https://nypost.com/2025/10/09/us-news/rep-andy...,179
200,2025-10-09-21-17-33 +0000,nypost,Zohran Mamdani refuses to credit Trump over Ga...,https://nypost.com/2025/10/09/us-news/zohran-m...,168
353,2025-10-09-11-51-45 +0000,nypost,Gazans join Israelis in chanting Trump’s name ...,https://nypost.com/2025/10/09/us-news/gazans-j...,161


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
39,198,2025-10-10-09-06-09 +0000,nypost,Why Trump wasn’t awarded Nobel Peace Prize for...,https://nypost.com/2025/10/10/us-news/why-trum...
174,107,2025-10-09-22-34-27 +0000,nypost,American military to oversee initial Gaza peac...,https://nypost.com/2025/10/09/us-news/american...
166,82,2025-10-09-22-51-00 +0000,wsj,The Justice Department has secured an indictme...,https://www.wsj.com/politics/justice-departmen...
123,51,2025-10-10-01-15-44 +0000,nyt,Federal Judge Temporarily Blocks Trump’s Natio...,https://www.nytimes.com/2025/10/09/us/judge-na...
206,48,2025-10-09-20-54-59 +0000,nypost,Jubilant Trump vows Gaza peace deal will ‘move...,https://nypost.com/2025/10/09/us-news/jubilant...
267,40,2025-10-09-17-37-51 +0000,nypost,Portland police union head welcomes fed suppor...,https://nypost.com/2025/10/09/us-news/portland...
101,39,2025-10-10-03-22-00 +0000,wsj,"As tensions rise on Capitol Hill, Republican a...",https://www.wsj.com/politics/policy/government...
296,32,2025-10-09-16-00-32 +0000,bbc,All Post Office Horizon victims entitled to fr...,https://www.bbc.com/news/articles/cg7dx7le0ddo...
49,31,2025-10-10-09-01-47 +0000,nypost,Venezuelan opposition leader María Corina Mach...,https://nypost.com/2025/10/10/world-news/venez...
285,31,2025-10-09-16-41-31 +0000,bbc,Texas court blocks death row inmate's executio...,https://www.bbc.com/news/articles/cn4lexk7wn7o...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
